## NDBC module
https://supermanzer.github.io/NDBC/html/NDBC.html

In [ ]:
!pip install NDBC
# !pip install windrose

In [ ]:
from NDBC.NDBC import DataBuoy
# import folium
import re
import matplotlib.pyplot as plt
from windrose import WindroseAxes, plot_windrose
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


In [ ]:
DB = DataBuoy()
DB.set_station_id('42020')
# DB.set_station_id('42091')

DB.get_station_metadata()
DB.station_info

In [ ]:
DB.station_search(distance=400)

[]

In [ ]:
DB = DataBuoy()
DB.set_station_id('42019')
DB.get_station_metadata()
latitude = float(re.findall(r"[-+]?\d*\.\d+|\d+", DB.station_info['lat'])[0])
longitude = - float(re.findall(r"[-+]?\d*\.\d+|\d+", DB.station_info['lon'])[0])
location_name = '42019'

map = folium.Map(location=[latitude, longitude], zoom_start=6)
folium.Marker([latitude, longitude], popup=location_name).add_to(map)

# Add markers for other stations returned by station_search
# for station in DB.station_search(distance=400):
for station in ['42002', '42019', '42020']:
    DB_ = DataBuoy()
    DB_.set_station_id(station)
    DB_.get_station_metadata()
    latitude = float(re.findall(r"[-+]?\d*\.\d+|\d+", DB_.station_info['lat'])[0])
    longitude = - float(re.findall(r"[-+]?\d*\.\d+|\d+", DB_.station_info['lon'])[0])
    location_name = station
    folium.Marker([latitude, longitude], popup=location_name).add_to(map)

map

In [ ]:
def process_buoy_data(station_id):
    DB = DataBuoy()
    DB.set_station_id(station_id)
    DB.get_data(months=range(1, 11), years=range(2010, 2024), datetime_index=True)

    df = DB.data['stdmet']['data']
    column_name_mapping = {
        'WDIR': 'wind_direction',
        'WSPD': 'wind_speed',
        'GST': 'wind_gust',
        'WVHT': 'wave_height',
        'DPD': 'dominant_wave_period',
        'APD': 'average_wave_period',
        'MWD': 'mean_wave_direction',
        'PRES': 'pressure',
        'ATMP': 'air_temperature',
        'WTMP': 'water_temperature',
        'DEWP': 'dewpoint',
        'VIS': 'visibility',
        'TIDE': '3hr_pressure_tendency',
    }

    df.rename(columns=column_name_mapping, inplace=True)

    df['Year'] = df.index.year
    df['Month'] = df.index.month
    excel_filename = f"Offshore_Wave_{station_id}_data.xlsx"
    df.to_excel(excel_filename, index=True)
    return df

# Example usage
df_42002 = process_buoy_data('42002')
df_42019 = process_buoy_data('42019')
df_42020 = process_buoy_data('42020')


In [ ]:
df.columns

In [ ]:
df['time'] = df.index

# fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
ax2b = ax2.twinx()

# Atmospheric Pressure (pressure)
ax1.plot(df['time'], df['wave_height'], color='black')
ax1.set_ylabel('wave_height [m]')

# Wind speed (wind_speed), gust (wind_gust), and direction (wind_direction)
ax2.plot(df['time'], df['wind_speed'], color='tab:orange')
ax2b.plot(df['time'], df['wind_direction'], color='tab:blue', linestyle='-')
ax2.set_ylabel('Wind Speed [m/s]')
ax2b.set_ylabel('Wind Direction [degrees]')

# Water temperature (water_temperature)
# ax3.plot(df['time'], df['water_temperature'], color='tab:brown')
# ax3.set_ylabel('Water Temperature [degC]')

plt.show()



In [ ]:
col_units = {'wind_direction': 'degrees',
                     'wind_speed': 'meters/second',
                     'wind_gust': 'meters/second',
                     'wave_height': 'meters',
                     'dominant_wave_period': 'seconds',
                     'average_wave_period': 'seconds',
                     'dominant_wave_direction': 'degrees',
                     'pressure': 'hPa',
                     'air_temperature': 'degC',
                     'water_temperature': 'degC',
                     'dewpoint': 'degC',
                     'visibility': 'nautical_mile',
                     '3hr_pressure_tendency': 'hPa',
                     'water_level_above_mean': 'feet',
                     'time': None}

In [ ]:
def remove_rows_with_missing_values(df, column1, column2):
    return df.dropna(subset=[column1, column2])
df_ = remove_rows_with_missing_values(df, "wind_direction", "wind_speed")

In [ ]:

def plot_windrose_subplots(data, *, direction, var, color=None, **kwargs):
    """wrapper function to create subplots per axis"""
    ax = plt.gca()
    ax = WindroseAxes.from_ax(ax=ax)
    plot_windrose(direction_or_df=data[direction], var=data[var], ax=ax, **kwargs)

In [ ]:
g = sns.FacetGrid(
    data=df_,
    # the column name for each level a subplot should be created
    col="Year",
    # place a maximum of 3 plots per row
    col_wrap=3,
    subplot_kws={"projection": "windrose"},
    sharex=False,
    sharey=False,
    despine=False,
    height=3.5,
)

g.map_dataframe(
    plot_windrose_subplots,
    direction="wind_direction",
    var="wind_speed",
    normed=True,
    # manually set bins, so they match for each subplot
    bins=(0.1, 3, 6, 9, 12, 15),
    calm_limit=0.1,
    kind="bar",
)

y_ticks = range(0, 17, 4)
for ax in g.axes:
    ax.set_legend(
        title="$m \cdot s^{-1}$", bbox_to_anchor=(1.15, -0.1), loc="lower right"
    )
    ax.set_rgrids(y_ticks, y_ticks)

# adjust the spacing between the subplots to have sufficient space between plots
plt.subplots_adjust(wspace=-0.2)

In [ ]:
df_ = remove_rows_with_missing_values(df, "mean_wave_direction", "wave_height")

g = sns.FacetGrid(
    data=df_,
    # the column name for each level a subplot should be created
    col="Year",
    # place a maximum of 3 plots per row
    col_wrap=3,
    subplot_kws={"projection": "windrose"},
    sharex=False,
    sharey=False,
    despine=False,
    height=3.5,
)

g.map_dataframe(
    plot_windrose_subplots,
    direction="mean_wave_direction",
    var="wave_height",
    normed=True,
    # manually set bins, so they match for each subplot
    bins=(0.1, 1, 2, 3, 4, 5),
    calm_limit=0.1,
    kind="bar",
)

y_ticks = range(0, 17, 4)
for ax in g.axes:
    ax.set_legend(
        title="$m$", bbox_to_anchor=(1.15, -0.1), loc="lower right"
    )
    ax.set_rgrids(y_ticks, y_ticks)

# adjust the spacing between the subplots to have sufficient space between plots
plt.subplots_adjust(wspace=-0.2)

In [ ]:
df_


In [ ]:
sns.displot(
    data=df_,
    x="wind_speed",
    col="Year",
    hue = "Month",
    kind="kde",
    col_wrap=3,
    height=3.5,
    facet_kws={'sharex': False, 'sharey': False},
    common_norm=False  # Normalize the KDE for each hue category separately
)


In [ ]:
sns.displot(
    data=df_,
    x="wind_speed",
    col="Year",
    hue = "Month",
    kind="ecdf",
    col_wrap=3,
    height=3.5,
    facet_kws={'sharex': False, 'sharey': False},
)


In [ ]:
df_42002.index[0], df_42002.index[-1]

In [ ]:
df_42019.index[0], df_42019.index[-1]

In [ ]:
df_42020.index[0], df_42020.index[-1]

In [ ]:
df_42002